In [74]:
import datetime
import json
import pathlib
import re

import numpy as np
import rioxarray

In [75]:
fn = "resize"
size = "50%"
url = "https://eoepca.org/media_portal/images/logo6_med.original.png"
xcengine_config = dict(
  workflow_id = "xceconvert-1"
)

In [76]:
ds = rioxarray.open_rasterio(url)

/home/pont/mambaforge/envs/xcube2505a/lib/python3.13/site-packages/rioxarray/_io.py:1143: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore


In [77]:
ds_small = ds.coarsen(x=2, y=2).mean().astype(np.uint8)

In [79]:
output_basename = re.sub("[.]png$", "", pathlib.Path(url).parts[-1]) + "-resize"
output_pngname = output_basename + ".png"
output_pngpath = pathlib.Path(output_pngname)
output_jsonname = output_basename + ".json"

In [80]:
ds_small.rio.to_raster(output_pngname)

In [81]:
now = datetime.datetime.now(datetime.UTC)
now_iso8601 = re.sub(r"\+00:00$", "Z", now.isoformat(timespec="milliseconds"))

In [83]:
item_stac = {
  "stac_version": "1.0.0",
  "id": f"{output_basename}-{now.timestamp():.9f}",
  "type": "Feature",
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [-180, -90],
        [-180, 90],
        [180, 90],
        [180, -90],
        [-180, -90]
      ]
    ]
  },
  "properties": {
    "created": now_iso8601,
    "datetime": now_iso8601,
    "updated": now_iso8601
  },
  "bbox": [-180, -90, 180, 90],
  "assets": {
    output_basename: {
      "type": "image/png",
      "roles": ["data"],
      "href": output_pngname,
      "file:size": output_pngpath.stat().st_size
    }
  },
  "links": [{
    "type": "application/json",
    "rel": "parent",
    "href": "catalog.json"
  }, {
    "type": "application/geo+json",
    "rel": "self",
    "href": output_jsonname
  }, {
    "type": "application/json",
    "rel": "root",
    "href": "catalog.json"
  }]
}

with open(output_jsonname, "w") as fh:
    json.dump(item_stac, fh, indent=2)

In [86]:
catalog_stac = {
  "stac_version": "1.0.0",
  "id": "catalog",
  "type": "Catalog",
  "description": "Root catalog",
  "links": [{
    "type": "application/geo+json",
    "rel": "item",
    "href": output_jsonname
  }, {
    "type": "application/json",
    "rel": "self",
    "href": "catalog.json"
  }]
}

with open("catalog.json", "w") as fh:
    json.dump(catalog_stac, fh, indent=2)